First we instantiate the simulator class we will use to generate our sequences.

In [1]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.detection import Detection

from stonesoup.types.track import Track
from stonesoup.types.hypothesis import SingleHypothesis

class simulator():

    def __init__(self, transition_model, measurement_model):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.transition_model= transition_model
        self.measurement_model = measurement_model        
    
    def _generate_ground_truth(self, prior, time_span):
        
        time_interval = time_span[1]-time_span[0]
        ground_truth = GroundTruthPath([prior])
        for k in range(1, len(time_span)):
            ground_truth.append(GroundTruthState(
                self.transition_model.function(ground_truth[k-1], noise=False, time_interval=time_interval),
                timestamp=time_span[k-1]))
        return ground_truth
    
    def _simulate_measurements(self, ground_truth):
        #Simulate Measurements
        measurements = []
        for state in ground_truth:
            measurement = self.measurement_model.function(state, noise=True)
            measurements.append(Detection(measurement,
                                          timestamp=state.timestamp,
                                          measurement_model=self.measurement_model))
        return measurements
    
    def generate_training_data(self, initial_state, time_span):
        """

        Parameters
        ----------
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        return ground_truth, measurements

    def simulate_track(self, predictor, updater, initial_state, prior, time_span):
        """

        Parameters
        ----------
        predictor : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        updater : :class:`~.Predictor`
            The Updater to be used.
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        #Initialize Loop Variables
        track = Track()
        for measurement in measurements:
            prediction = predictor.predict(prior, timestamp=measurement.timestamp)
            hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
            posterior = updater.update(hypothesis)
            track.append(posterior)
            prior = track[-1]
        return ground_truth, track

In [2]:
"""
@author: John Hiles
"""
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime

from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import State, GaussianState, EnsembleState

from stonesoup.models.transition.linear import (CombinedLinearGaussianTransitionModel,
                                                ConstantVelocity)
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.updater.ensemble import (EnsembleUpdater, EnsembleSqrtUpdater)
from stonesoup.updater.kalman import KalmanUpdater, ExtendedKalmanUpdater
from stonesoup.predictor.ensemble import EnsemblePredictor
from stonesoup.predictor.kalman import KalmanPredictor, ExtendedKalmanPredictor
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange



"""
    This script represents the code used to gather the data used in [PAPER HERE].
    
    This repository is structured such that different stone soup algorithms 
    can be run rapidly. Hopefully I've made it modular enough to 
    allow swapping of things like algorithms, and "experiments" by replacing
    the desired transition and measurement models.
    
    The simulator class requires a transition and 
    measurement model, then the simulate_track method accepts a Stone Soup
    Predictor, Updater, ground truth initial state, initial state for the
    chosen algorithm, and a span of time which the simulation takes place over.
    This time span should be an evenly spaced datetime.datetime list.
    
    The simulator then, is used to gather "Track" instances, and with a list 
    of tracks, RMSE can then be calculated.
"""

i = 30
num_vectors = i*5

"""
    Here, we get our initial variables for simulation. For this, we are just
    using a time span of 60 time instances spaced one second apart.
"""

timestamp = datetime.datetime(2021, 4, 2, 12, 28, 57)
tMax = 120
dt = 1
tRange = tMax // dt
plot_time_span = np.array([dt*i for i in range(tRange)])

time_span = np.array([timestamp + datetime.timedelta(seconds=dt*i) for i in range(tRange)])

monte_carlo_iterations = 10


"""
Here we instantiate our transition and measurement models. These are the 
same models used in the StoneSoup Kalman Filter examples.
"""

q_x = 0.05
q_y = 0.05
sensor_x = 50  # Placing the sensor off-centre
sensor_y = 0

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x), 
                                                          ConstantVelocity(q_y)])
measurement_model = CartesianToBearingRange(
ndim_state=4,
mapping=(0, 2),
noise_covar=np.diag([np.radians(0.2), 1]),  # Covariance matrix. 0.2 degree variance in
# bearing and 1 metre in range
translation_offset=np.array([[sensor_x], [sensor_y]])  # Offset measurements to location of
# sensor in cartesian.
)


"""
Here we instantiate the simulator with our transition and measurement model.
This class is capable of generating sets of ground truth points, and simulate
measurements for our recursive filters.
"""

nonlinear_simulator = simulator(transition_model=transition_model,
                      measurement_model=measurement_model)

"""
To get our data, we will run three simulations with varying values for 
num_vectors, which corresponds to M in our paper. The number of vectors in the
ensemble.

Structurally, we have two loops. The outer loop determines how many vectors
are in our ensemble, the inner loop runs the simulation to gather the data.
"""

initial_ground_truth = State(state_vector=StateVector([0, 1, 0, 1]),
                             timestamp = time_span[0])
import torch

ground_truth, measurements = nonlinear_simulator.generate_training_data(initial_ground_truth, time_span)


Now with our simulator we need to instantiate the Pytorch Dataset.

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis

#class sequence_generator(Dataset):

class dataset_2D_bearing_range(Dataset):
    #Range Bearing 2D Dataset Sequence Packer

    def __init__(self, dataset, ground_truth=None, measurements=None):
        if dataset == None:   
            gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
            gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
            ms = np.array([m.state_vector for m in measurements]).squeeze().T
            ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
            self.dataset = torch.unsqueeze(torch.cat((gt,ms)),dim=0)
        else:
            self.dataset = dataset
            
    
    def append_to_dataset(self, ground_truth, measurements):
        """
        Parameters
        ----------
        ground_truth : :class:`~.list`
            A list of Stonesoup States.
        measurements : :class:`~.measurements`
            The list of Stonesoup Measurements to be used.
        """
    
        gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
        gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
        ms = np.array([m.state_vector for m in measurements]).squeeze().T
        ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
        new_entry = torch.cat((gt,ms),dim=0)

        self.dataset = torch.cat((self.dataset,torch.unsqueeze(new_entry,dim=0)),dim=0)

    def __getitem__(self, idx):
        return self.dataset[idx,0:2].T, self.dataset[idx,0:4].T

    def __len__(self):
        return self.dataset.shape[0]



cuda


Now we need to simulate and add to our dataset.

In [4]:
training_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)

training_dataset_len = 1000

for i in range(training_dataset_len-1):
    ground_truth, measurements = nonlinear_simulator.generate_training_data(initial_ground_truth, time_span)
    training_dataset.append_to_dataset(ground_truth, measurements)

print(training_dataset.__len__())
#print(training_dataset.__getitem__(2)[0].shape)
#print(training_dataset.__getitem__(2)[1].shape)

trainloader = DataLoader(training_dataset)

testing_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)
for i in range(int((training_dataset_len)/4 -1)):
    ground_truth, measurements = nonlinear_simulator.generate_training_data(initial_ground_truth, time_span)
    testing_dataset.append_to_dataset(ground_truth, measurements)
testloader = DataLoader(testing_dataset)
print(testing_dataset.__len__())


1000
250


Turn Sequences into dataset

In [5]:
import torch.nn as nn
import torch.nn.functional as F

input_size = 2
hidden_size = 4
num_layers = 1
nonlinearity = 'tanh'
output_size = 4
# input_size, hidden_size, num_layers=1, nonlinearity='tanh', bias=True, batch_first=False, dropout=0.0, bidirectional=False, device=None, dtype=None
model = torch.nn.RNN(input_size, hidden_size, num_layers=num_layers, device=device)

In [6]:
criterion = nn.MSELoss()
# this optimizer will do gradient descent for us
# experiment with learning rate and optimizer type
learning_rate = 0.01;
# note that we have to add all weights&biases, for both layers, to the optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
# we add a learning rate scheduler, which will modify the learning rate during training
# will initially start low, then increase it ("warm up"), and then gradually decrease it
n_epochs = 100;
batch_size = 120;
num_updates = n_epochs*int(np.ceil(len(trainloader.dataset)/batch_size))
print(num_updates)
warmup_steps=10;
def warmup_linear(x):
    if x < warmup_steps:
        lr=x/warmup_steps
    else:
        lr=max( (num_updates - x ) / (num_updates - warmup_steps), 0.)
    return lr;
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_linear);

900


In [7]:
n_epochs = 25;
hidden_state = torch.zeros((1, batch_size, hidden_size),device=device,requires_grad = True)

#hidden_state.type(torch.float)
for i in range(n_epochs):
    for j, data in enumerate(trainloader):

        inputs, labels = data

        #For loop on sequence
        #for t in range()

        optimizer.zero_grad();
        #forward phase - predictions by the model
        #print(inputs.shape)
        outputs, hidden_state = model(inputs, hidden_state);
        #forward phase - risk/loss for the predictions
        risk = criterion(outputs.squeeze(), labels.squeeze());

        # calculate gradients
        risk.backward(retain_graph=True);

        # take the gradient step
        optimizer.step();
        #scheduler.step();

        batch_risk=risk.item();
    with (torch.no_grad()):
      for j, data in enumerate(testloader):
          inputs, labels = data
          inputs=inputs.to(device);
          labels=labels.to(device);
          outputs, hidden_state = model(inputs);
          risk = criterion(outputs.squeeze(), labels.squeeze());

    print(i, risk)

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [4, 2]] is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [8]:
test_tensor = torch.ones([3,6,120])

data_tensor = test_tensor[1,0:2]
label_tensor = test_tensor[1,1:-1]
    

#print(label_tensor)

#print(data_tensor)
print(test_tensor[1].shape)
print(data_tensor.shape)
print(label_tensor.shape)

print(test_tensor.shape)

torch.Size([6, 120])
torch.Size([2, 120])
torch.Size([4, 120])
torch.Size([3, 6, 120])
